# PPG analysis

In [73]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import os

from paradigma.preprocessing_config import PPGPreprocessingConfig, IMUPreprocessingConfig
from paradigma.ppg_preprocessing import scan_and_sync_segments, preprocess_ppg_data
from paradigma.heart_rate_analysis_config import HeartRateFeatureExtractionConfig, HeartRateClassificationConfig
from paradigma.heart_rate_analysis import extract_signal_quality_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
# Cell has the tag 'parameters'
path_to_data =  '../../../tests/data'
path_to_classifier = '../../../src/paradigma/ppg/classifier'

quality_classifier_path = os.path.join(path_to_classifier, "LR_PPG_quality.pkl")
path_to_sensor_data = os.path.join(path_to_data, '1.sensor_data')
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', 'ppg')
path_to_quality_features = os.path.join(path_to_data, '3.extracted_features', 'ppg')
path_to_signal_quality = os.path.join(path_to_data, '4.predictions', 'ppg')
path_to_hr_estimate = os.path.join(path_to_data, '5.quantification', 'ppg')

In [78]:
ppg_config = PPGPreprocessingConfig()
imu_config = IMUPreprocessingConfig()
metadatas_ppg, metadatas_imu = scan_and_sync_segments(os.path.join(path_to_sensor_data, 'ppg'),
                                                       os.path.join(path_to_sensor_data, 'imu'))
preprocess_ppg_data(metadatas_ppg[0], metadatas_imu[0],
                    path_to_preprocessed_data,
                    ppg_config, imu_config)

Shape of the original data: (64775, 2) (72947, 4)
Shape of the overlapping segments: (64775, 2) (64361, 4)


In [ ]:
config = HeartRateFeatureExtractionConfig()
extract_signal_quality_features(path_to_preprocessed_data, quality_classifier_path, path_to_quality_features, config)

In [ ]:
#TODO: Not fully implemented
config = HeartRateClassificationConfig()
signal_quality_classification(path_to_quality_features, quality_classifier_path, path_to_signal_quality, config)

In [ ]:
#TODO: Not implemented
# config = HeartRateEstimationConfig()
# estimate_heart_rate(path_to_signal_quality, path_to_hr_estimate, config)